# 概念分群[二]_定義概念群的幾個工具方法
以下方法是用來定義概念群時使用

LastModified : 20160117_2147

0.選定要使用的W2V Model (只能擇一，頗占記憶體)

In [1]:
import gensim
#1__WIKI
model = gensim.models.Word2Vec.load("W2V_Wiki_Trastd/wiki.zh.sim.seg.trastd.model")

#2__搜狐新闻数据
#model = gensim.models.Word2Vec.load('W2V_SogouCS/SogouCS_Combine.model')

#3__全网新闻数据
#model = gensim.models.Word2Vec.load('W2V_SogouCA/SogouCA_Combine.model')

# [1] 

    看_撈出的關鍵字_與_哪些詞(基於訓練資料ex.wiki...)_具有高相似度 (門檻自訂)

    可看出撈出的關鍵字中，與W2V中相似度最高的詞語是哪些，進而做為定義概念群中詞語的依據

使用範例：

    FindHighSim(Corpus,SimLevel)
    
    Corpus  : [字串]。[檔名]。存取關鍵字的TXT檔
    SimLevel: [浮點數]。[0~1]。相似度門檻

使用結果：

    會印出 :「關鍵字」-「相似字」-「相似度」( 每行 )

In [2]:
def FindHighSim(Corpus,SimLevel):
    x = set()
    b = set()
    for a in open(Corpus,'r'):
        a = a.strip().decode('utf-8')
        x.add(a)
    for c in x:
        try:
            result = model.most_similar(c)
            if result[0][1] > SimLevel:
                print c , result[0][0] , result[0][1]
        except:
            b.add(a)

In [3]:
FindHighSim('NE_SciTfidf-30-20160114.txt',0.75)

拓撲 同胚於 0.767044007778
佛教 漢傳 0.778333961964
猴洞 坑子 0.788516581059
炎熱 寒冷 0.778002917767
寧夏 甘肅 0.772675275803
神祇 主神 0.751127779484
接待員 鍾鈺精 0.762435138226
攝影 剪接 0.769067049026
評論 影評 0.755247294903
蔣中正 蔣介石 0.789977490902
夏季 冬季 0.834889173508
觀劇 網易娛樂 0.808738291264
蔣介石 蔣中正 0.789977550507
小食 小吃 0.820101678371
航班 包機 0.756068348885
昆明街 西寧南路 0.783462166786
佛像 泥塑像 0.762733519077
平溪 瑞芳 0.776917219162
幽暗 mirkwood 0.751509010792
捲菸廠 紅塔集團 0.775149643421
艦橋 右舷 0.755797803402
交響樂團 管弦樂團 0.793657422066
河流 下游 0.765522480011
航母 航空母艦 0.756751954556
北車 南車 0.783483564854
喜歡 討厭 0.815701305866
大使館 駐 0.780441045761
瑞芳 平溪 0.776917159557
寺院 藏傳佛教 0.79683470726
祭祀 祭拜 0.773009419441
遺址 新石器 0.823991775513
秋季 春季 0.772646844387
平溪鄉 三貂堡 0.77661049366
菁桐 望古 0.796829819679


# [2] 

    定義概念群 : 準則為「"組內"相似度高」&「"組間"相似度低」

    比較概念內詞語的相似度(InSim) , 與每兩概念間的相似度(OutSim) 
    
    來找出InSim不夠高，或是OutSim不夠低的概念，加以對概念內的詞語做調整
 
使用範例：

    CompareConcept(Dic,InSim,OutSim)
    
    Dic   :[字典]。概念字典。
           格式為：Key  : 概念的名稱(僅命名,無實質意義)
           　　　　Value: [陣列],內容是可以代表該概念的詞語
    
    InSim :[浮點數]。組內相似度。
           EX.通常設0.0，表示把"每一組"的組內相似度都印出來。
           若組內的相似度過低，就可以找是哪一個詞影響到了整體的相似度，把它拿掉。
           
    OutSim:[浮點數]。組間相似度。
           EX.若設為0.15，就是認為組間相似度達0.15就算太像，此時組間相似度超過0.15
           的兩個組就會被印出來。
           
使用結果：

    會回傳３個PART
    
    PART1　組內相似度
    
    PART2　組間相似度
    
    PART3　組內最不像的(離群值)
           此為W2V內建的方法,會傳回每個概念內「與詞語間最不相似的一個詞」
           用途：當組內相似度不夠高時，不知道要去除哪個詞，來提高組內相似度。
                    (EX 覺得[輕鬆]概念的組內相似度不夠高,使用此法,
                    結果顯示:[輕鬆]組內最不相似的詞是"美食",
                    就可以把"美食"從[輕鬆]裡拿掉,藉以提高[輕鬆]的組內相似度)
                 此結果僅為參考,去除後組內相似度並不見得會提高;
                 要提高也可利用前者的方法 --FindHighSim() 去新增概念內的詞語

In [4]:
def CompareConcept(Dic,InSim,OutSim):
    import operator
    import itertools
    
    print '組內相似度\n---'
    SimInside = {}
    for i in Dic:
        cnt = 0
        percent = 0.0
        #print i,'--'
        Test = i
        for w in Dic[Test]:
            for o in Dic[Test]:
                cnt += 1
                percent += model.similarity(w.decode('utf-8'),o.decode('utf-8'))
                #print i.decode('utf-8'),\
                #      o.decode('utf-8'),\
                #      model.similarity(i.decode('utf-8'), o.decode('utf-8'))
        #print '--- Ave :',percent/cnt  
        SimInside[i] = percent/cnt 

    Rank = sorted(SimInside.iteritems(),key=operator.itemgetter(1),reverse=True)
    for z in Rank:
        if z[1] > InSim:
            print z[0],z[1]

    print '\n\n組間相似度\n---'
    SimOutside = {}
    li = []
    for e in Dic:
        li.append(e.decode('utf-8'))
    for q in list(itertools.combinations(li,2)):
        cnt2 = 0
        percent2 = 0.0
        for x in Dic[q[0].encode('utf-8')]:
            for y in Dic[q[1].encode('utf-8')]:
                cnt2 += 1
                percent2 += model.similarity(x.decode('utf-8'),y.decode('utf-8'))
        #print q[0],q[1],percent2/cnt
        SimOutside[' '.join(q)] = percent2/cnt

    Rank2 = sorted(SimOutside.iteritems(),key=operator.itemgetter(1),reverse=True)
    for d in Rank2:
        if d[1] > OutSim:
            print d[0],d[1]


    print '\n\n組內最不像的(離群值)\n---'
    for u in Dic:
        print u,model.doesnt_match(' '.join(Dic[u]).decode('utf-8').split())

＠自定義的概念字典

In [5]:
Dic = {
'藝文':['藝術','美術','戲劇','音樂'],\
    
'信仰':['信仰','祭祀','祭拜','膜拜','信眾','信奉'],\
    
'美食':['美食','爽口','美味','好吃'],\
'新奇':['新奇','新穎','獵奇','獨特','驚喜'],\

'輕鬆':['輕鬆','休閒','遊憩','悠閒','休憩','消遣','愜意','放鬆'],\
'遊玩':['遊玩','好玩','遊樂'],\
'快樂':['快樂','愉快','開心','歡樂','幸福','美好'],\
'有趣':['有趣','趣味'],\

'脫俗':['脫俗','幽雅','清幽','優雅','氣質','心曠神怡'],\
'樂活':['環保','綠化','清新'],\

'安靜':['安靜','寧靜'],\
'乾淨':['乾淨','整潔','潔淨'],\

'懷舊':['懷舊','復古','情調'],\
'古老':['古老','悠久'],\

'髒亂':['骯髒','很髒','髒亂','低劣','灰塵','破舊'],\

'購物':['商圈','百貨','商場'],\
'特產':['特產','紀念品'],\

'運動':['運動','健康'],\

'漂亮':['漂亮','可愛','美麗','優美'],\
'文青':['清新','優雅','氣質'],\

'壯觀':['雄偉','壯觀','壯麗']
}

In [6]:
CompareConcept(Dic,0,0.15)

組內相似度
---
古老 0.818809681714
文青 0.711611056913
壯觀 0.706678632056
有趣 0.69204862026
安靜 0.67246421226
運動 0.616173825221
購物 0.615059877651
信仰 0.611419907947
美食 0.589275201898
特產 0.587917442123
乾淨 0.585851257683
懷舊 0.584266524582
漂亮 0.562307795312
藝文 0.499157189805
脫俗 0.486872799178
樂活 0.485786980765
快樂 0.465091752659
遊玩 0.451244988725
髒亂 0.449564396734
新奇 0.437149979195
輕鬆 0.344181888323


組間相似度
---
脫俗 輕鬆 0.152523054568


組內最不像的(離群值)
---
古老 古老
遊玩 好玩
新奇 驚喜
懷舊 情調
乾淨 潔淨
壯觀 壯麗
漂亮 優美
信仰 信奉
運動 運動
脫俗 心曠神怡
髒亂 低劣
樂活 清新
安靜 安靜
快樂 歡樂
美食 爽口
特產 特產
購物 商場
有趣 趣味
文青 清新
藝文 戲劇
輕鬆 休憩


# [3] 
    
    詞語轉移概念 -> 一詞語若與原概念不相似, 可用此法尋求可以放到哪個其他的概念裡

　EX.
    
    情況A "臭味"原本在[輕鬆]裡, 卻被發現此詞是拉低[輕鬆]組內相似度的罪魁禍首 
    情況B "臭味"是我從關鍵詞中, 覺得可以被當作概念詞語的關鍵詞 
     -->發生前兩種情況, 卻不知道該歸類在哪一個概念比較適合的時候, 可以用此法來判定
        "臭味"與現有的所有概念詞中, 哪個最相像, 並顯示相似度 .若夠高, 可試著加入

使用範例：

    findSim(word)

    word   :[字串]。鍵入想查詢的關鍵詞。

In [7]:
def findSim(word):
    import operator
    dicli = {}
    for a in Dic:
        for b in Dic[a]: 
            dicli['Concept:'+a+" ->Item:"+b] = model.similarity(b.decode('utf-8'),word.decode('utf-8'))

    print '最相近的概念'
    print \
    sorted(dicli.iteritems(),key=operator.itemgetter(1),reverse=True)[1][0],\
    '　',\
    sorted(dicli.iteritems(),key=operator.itemgetter(1),reverse=True)[2][1]

In [8]:
findSim('臭味')

最相近的概念
Concept:髒亂 ->Item:很髒 　 0.445742046467


＠最基本的找兩詞語相似度

In [9]:
print model.similarity(u'情調',u'情調')

1.0
